The Swap test is a simple quantum circuit which, given two states, allows to compute how much do they differ from each other.

## Import modules

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from register import QuantumRegister
from gate import QuantumGate
from utils import plot_counts
from program_parser import parse_program

import numpy as np

from scipy.optimize import minimize

import math
import random

## Solving first part

1. Provide a variational (also called parametric) circuit which is able to generate the most general 1 qubit state. By most general 1 qubit state we mean that there exists a set of the parameters in the circuit such that any point in the Bloch sphere can be reached. Check that the circuit works correctly by showing that by varying randomly the parameters of your circuit you can reproduce correctly the Bloch sphere.

Any quantum state can be represented on a bloch sphere by a rotation around the Y-axis followed by a rotation around the Z-axis. Since we are restricted to states on a Bloch sphere, we are ignoring the global phase.

In [3]:
def general_state(circuit, index, theta, phi):
    circuit.add_gate(QuantumGate('Ry', theta), [index])
    circuit.add_gate(QuantumGate('Rot', phi), [index])
    circuit.apply()

In [4]:
reg = QuantumRegister(1)

print(reg.get_statevector())

theta = math.pi / 2
phi = math.pi / 2

general_state(reg, 0, theta, phi)

print(reg.get_statevector())

reg.measure(100)

[1.+0.j 0.+0.j]
[0.70710678+0.j         0.        +0.70710678j]


{'0': 44, '1': 56}

## Solving second part

2. Use the circuit built in step 1) and, using the SWAP test, find the best choice of your parameters to reproduce a randomly generated quantum state made with 1 qubit.

#### First, generate a random state.

In [5]:
theta_r = random.uniform(0.0, math.pi)
phi_r = random.uniform(0.0, 2*math.pi)

theta_r, phi_r

(0.3728611729884365, 5.0090304310690055)

#### Check the statevector of the state

In [6]:
tmp = QuantumRegister(1)
general_state(tmp, 0, theta_r, phi_r)
tmp.get_statevector()

array([0.98267209+0.j        , 0.0541804 -0.17725699j])

#### Run the optimisation routine

In [7]:
circ = [   
    { "gate": "h", "target": [0] },
    { "gate": "ry", "params": { "theta": theta_r}, "target": [2] },
    { "gate": "rot", "params": { "theta": phi_r}, "target": [2] },
    { "gate": "ry", "params": { "theta": "global_1"}, "target": [1] },
    { "gate": "rot", "params": { "theta": "global_2"}, "target": [1] },
    {"gate":"cswap", "target": [0, 1, 2]},
    { "gate": "h", "target": [0] }
]

circ = parse_program(circ)

reg = QuantumRegister(3)

In [8]:
def run_circuit(params, shots):
    reg.reset()
    reg.run_program(circ, { "global_1": params[0], "global_2": params[1] })
    
    return reg.measure(shots, [0])

In [9]:
def objective_function(params):
    measurements = 10000
    counts = run_circuit(params, measurements)

    zeros = counts['0']

    ratio = zeros / measurements
    
    return 1 - 2 * ratio

In [10]:
params = np.array([1.5708, 1.5708])

In [11]:
%%capture
minimum = minimize(objective_function, params, method="Powell", tol=1e-7)

In [12]:
minimum

   direc: array([[1., 0.],
       [0., 1.]])
     fun: array(-1.)
 message: 'Optimization terminated successfully.'
    nfev: 137
     nit: 2
  status: 0
 success: True
       x: array([-0.37171107,  1.87039125])

In [13]:
theta_a, phi_a = minimum.x

#### Check the validity of the solution

In [14]:
approx = QuantumRegister(1)
general_state(approx, 0, theta_a, phi_a)
print('Original:\t', tmp.get_statevector())
print('Approximated:\t', approx.get_statevector())

mse = (np.square(tmp.get_statevector() - approx.get_statevector())).mean()
print('MSE:\t', np.round(mse, 5))

Original:	 [0.98267209+0.j         0.0541804 -0.17725699j]
Approximated:	 [0.98277852+0.j         0.0545369 -0.17655626j]
MSE:	 (-0+0j)


## Solving third part

3. Suppose you are given with a random state, made by N qubits, for which you only know that it is a product state and each of the qubits are in the state | 0 > or | 1>. By product state we mean that it can be written as the product of single qubit states, without the need to do any summation. For example, the state |a> = |01> is a product state, while the state |b> = |00> + |11> is not.

Perform a qubit by qubit SWAP test to reconstruct the state. This part of the problem can be solved via a simple grid search.

In [15]:
n = 5

init_state = ''
for i in range(n):
    tmp = random.randint(0, 10)
    
    if tmp > 4:
        init_state += '0'
    else:
        init_state += '1'
init_state

'00110'

In [16]:
# Create the circuit
reg = QuantumRegister(2 * n + 1)

In [17]:
def initialise_circuit(state_string, circuit):
    for i in range(n):
        if state_string[i] == '0':
            circuit.initialise_qubit(i + 1, QuantumRegister.zero)
        else:
            circuit.initialise_qubit(i + 1, QuantumRegister.one)

In [18]:
def detect_state(circuit):
    result = ''
    
    for i in range(n):
        reg.reset()
        
        initialise_circuit(init_state, reg)
        
        circ = [
            { "gate": "h", "target": [0] },
            { "gate": "cswap", "target": [0, i + 1, (i + 1) + n]},
            { "gate": "h", "target": [0] }
        ]
        
        circ = parse_program(circ)
        
        reg.run_program(circ)
        
        counts = reg.measure(10, [0])
        
        found = False
        for key in counts.keys():
            if key.startswith('1'):
                result += '1'
                found = True
                break
        
        if not found:
            result += '0'
    
    return result

In [ ]:
state = detect_state(reg)

In [20]:
state, init_state

('00110', '00110')